In [1]:
# !conda init
# !exec $SHELL
!conda activate clipenv


CondaError: Run 'conda init' before 'conda activate'



In [ ]:
import argparse
import os
os.environ["WORLD_SIZE"] = "1"
import random
import Logger
import torch
torch.autograd.set_detect_anomaly(True)
# from test import test
import mobileclip

In [ ]:

def get_args_parser():
    parser = argparse.ArgumentParser()
    parser.add_argument('--pretrained_path',type=str)
    parser.add_argument('--model', default='mobileclip_s0', type=str)
    parser.add_argument('--model_cfg', default='mobileclip_s0_for_binary', type=str)
    parser.add_argument('--input_size', default=128, type=int)
    parser.add_argument('--test', action='store_true')
    return parser
parser = get_args_parser()
args = parser.parse_args(args=[])
log = Logger.Logger()
from datetime import datetime
title = datetime.now().strftime('%b%d_%H-%M-%S')
if not args.test:
    args.pretrained_path = os.path.join(args.pretrained_path, args.model) + '.pt'
print('args.pretrained_path:', args.pretrained_path)




In [ ]:
args.pretrained_path='./mobileclip_mh/weights/checkpoint.pth'
model, _, preprocess = mobileclip.create_model_and_transforms(args, args.model, pretrained=args.pretrained_path, reparameterize=False)
tokenizer = mobileclip.get_tokenizer(args.model_cfg)
label_to_caption = [
                    "This is an example of a real face",
                    "This is a bonafide face",
                    "This is a real face",
                    "This is how a real face looks like",
                    "a photo of a real face",
                    "This is not a spoof face",
                    "This is an example of a spoof face",
                    "This is an example of an attack face",
                    "This is not a real face",
                    "This is how a spoof face looks like",
                    "a photo of a spoof face",
                    "a printout shown to be a spoof face",
                    ]
# images in skimage to use and their textual descriptions
descriptions = {
    "astronaut": "a portrait of an astronaut with the American flag",
}

In [9]:
import os
import skimage
from PIL import Image
import torch

# image = self.preprocess(Image.open(image_path).convert('RGB'))
# label = self.labels[idx]
original_images = []
images = []
texts = []
for filename in [filename for filename in os.listdir(skimage.data_dir) if filename.endswith(".png") or filename.endswith(".jpg")]:
    name = os.path.splitext(filename)[0]
    if name not in descriptions:
        continue
    image = Image.open(os.path.join(skimage.data_dir, filename)).convert("RGB")
    original_images.append(image)
    images.append(preprocess(image))
    caption = random.choice(label_to_caption)
    text = tokenizer(caption, return_tensors="pt").squeeze()
    texts.append(text)
    


In [10]:

model = model.cuda()
model.eval()
image_features = model.image_encoder(images[0].unsqueeze(0).cuda())
text_features = model.text_encoder(texts[0].unsqueeze(0).cuda())

In [ ]:
output = './tflite_tmp'
os.makedirs(output, exist_ok=True)
torch.onnx.export(model.text_encoder, texts[0].unsqueeze(0).cuda(), os.path.join(output,"mobileclip-text-vit-32_v2.onnx"), verbose=False, opset_version=12, input_names=['images'],
                    output_names=['output'],
                    dynamic_axes=None)

In [19]:

# torch.onnx.export(model.image_encoder, images[0].unsqueeze(0).cuda(), os.path.join(output,"mobileclip-image-vit-32.onnx"), export_params=True, opset_version=14, do_constant_folding=True, input_names = ['input'], output_names = ['output'], dynamic_axes={'input' : {0 : 'batch_size'}, 'output' : {0 : 'batch_size'}})
# torch.onnx.export(model.image_encoder, images[0].unsqueeze(0).cuda(), os.path.join(output,"mobileclip-image-vit-32.onnx"), export_params=True, opset_version=14, input_names = ['input'], output_names = ['output'])
torch.onnx.export(model.image_encoder, images[0].unsqueeze(0).cuda(), os.path.join(output,"mobileclip-image-vit-32_v2.onnx"), verbose=False, opset_version=12, input_names=['images'],
                    output_names=['output'],
                    dynamic_axes=None)
     

In [20]:
# Attempt at quantizing model to uint8 (doesn't seem to work? no errors, but onnx file is same size)
# Reference: https://github.com/minimaxir/imgbeddings/blob/36fb4d7ac6b82694d109cef6f887d4cb9c49da0f/imgbeddings/models.py#L94
# Here's the model the above code generates: https://huggingface.co/minimaxir/imgbeddings/blob/main/patch32_v1.onnx
# Here's a demo of the above ONNX model with ORT Web: https://jsbin.com/nupehazaju/edit?html,output  <-- seems to work, but this model doesn't have the projection head that squashes 768 vec to 512 elements (so can be compared to text embeddings of same length)
# !pip install onnxruntime
# !pip install onnx
from onnxruntime.quantization import quantize_dynamic, QuantType


In [ ]:

# quantize_dynamic(os.path.join(output,"mobileclip-image-vit-32.onnx"), os.path.join(output,"mogileclip-image-vit-32-uint8.onnx"), weight_type=QuantType.QUInt8, extra_options={"MatMulConstBOnly":False}) # I added the MatMulConstBOnly as a guess due to warnings that it outputs without it
# quantize_dynamic(os.path.join(output,"mobileclip-text-vit-32.onnx"), os.path.join(output,"mogileclip-text-vit-32-uint8.onnx"), weight_type=QuantType.QUInt8, extra_options={"MatMulConstBOnly":False}) # I added the MatMulConstBOnly as a guess due to warnings that it outputs without it
quantize_dynamic(os.path.join(output,"mobileclip-text-vit-32.onnx"), os.path.join(output,"mogileclip-text-vit-32-uint8_int8.onnx"), weight_type=QuantType.QInt8, extra_options={"MatMulConstBOnly":False}) # I added the MatMulConstBOnly as a guess due to warnings that it outputs without it

In [ ]:
# !pip install git+https://github.com/onnx/onnx-tensorflow.git
#export tflite using onnx
!onnx-tf convert -i tflite_tmp/mobileclip-text-vit-32_v2.onnx -o tflite_tmp/mobileclip-text-vit-32-tf_v2
!onnx-tf convert -i tflite_tmp/mobileclip-image-vit-32_v2.onnx -o tflite_tmp/mobileclip-image-vit-32-tf_v2

In [25]:
# !pip install tensorflowjs
import tensorflow as tf

In [ ]:

# image encoder:
converter = tf.lite.TFLiteConverter.from_saved_model(os.path.join(output,"mobileclip-image-vit-32-tf_v2"))
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS] # This line is needed because: https://github.com/tensorflow/tfjs/issues/5844
tflite_model = converter.convert()
with open(os.path.join(output,'mobileclip-image-vit-32_v2.tflite'), 'wb') as f:
  f.write(tflite_model)


In [ ]:
# text encoder:
converter = tf.lite.TFLiteConverter.from_saved_model(os.path.join(output,"mobileclip-text-vit-32-tf_v2_float"))
# converter = tf.lite.TFLiteConverter.from_saved_model(os.path.join(output,"mobileclip-text-vit-32-tf_v2"))
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS] # This line is needed because: https://github.com/tensorflow/tfjs/issues/5844
tflite_model = converter.convert()
with open(os.path.join(output,'mobileclip-text-vit-32-tf_v2_float.tflite'), 'wb') as f:
  f.write(tflite_model)
